# Using the Scheduler

In [2]:
from qiskit import QuantumCircuit

circ = QuantumCircuit(1, 1)
# circ = QuantumCircuit(2, 2)
circ.h(0)
# circ.cx(0, 1)
# circ.measure([0, 1], [0, 1])
circ.measure([0], [0])

circ.draw()

In [ ]:
from qiskit import IBMQ

provider = IBMQ.load_account()
backend = provider.get_backend(open_pulse=True)

In [ ]:
from qiskit.compiler import transpile, schedule as build_schedule

schedule = build_schedule(transpile(circ, backend), backend)

In [ ]:
from qiskit.pulse import Gaussian, DriveChannel, Schedule
custom_q0_h_schedule = Schedule(Gaussian(duration=128, amp=0.045, sigma=8)(DriveChannel(0)))

inst_map = backend.defaults().instruction_schedule_map
inst_map.add('h', 0, custom_q0_h_schedule)


schedule = build_schedule(circ,
                          backend,
                          inst_map=inst_map)

